# Load modules

In [ ]:
# load modules
from rich import print
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# from directory_structure import Tree
import sys, os
import os.path as path
import glob
import re
import pickle

modules = path.abspath(path.join(os.getcwd(), "helpers"))
print(modules)
sys.path.insert(0, modules)

from import_files import *
from preprocessing import *
from plotting import *

%matplotlib inline

# Load data, rename columns, save/load to pickles to increase speed (and remove channels)

In [ ]:
# get list of files and pickles
####################################

file_lst = glob.glob("Data/" + animal + "/*.txt")
print(file_lst)

pickle_lst = glob.glob("pickles/" + animal + "*.pckl")
print(pickle_lst)

# make df names
####################################

df_name_lst = []
for x in file_lst:
    if 'eed' in x:
        df_name_lst.append('df_feed')
    if 'ase' in x:
        df_name_lst.append('df_base')

# load pickles or make pickles
####################################

if len(file_lst) == len(pickle_lst):
    print("import pickles")
    for x, y in zip(pickle_lst, df_name_lst):
        print(y)
        df_x = Import.open_pickle(x)
        exec(y + " = df_x")
        print()
else:
    print("import CSVs and make pickles")
    for x, y in zip(file_lst, df_name_lst):
        print(y)
        df_x = Import.csv_import_to_pandas(file_path=x, df_name=y, print_df=False)
        df_x.columns = col_names
        # drop channel
        df_x = df_x[chan_to_keep]
        print(df_x)
        # store pickle
        # create directory if it does not exist
        file_path = 'pickles/'
        if not os.path.exists(file_path):
            os.mkdir(file_path)
        Import.store_pickle(obj=df_x, file_path='pickles/' + animal + '_' + y +'.pckl')
        exec(y + " = df_x")
        print()

## Determine percentage missing values

In [ ]:
for x in df_name_lst:
    print(x)
    df1 = locals()[x]
    nans_lst, zeros_lst = Preprocessing.percent_nans_zeros_tolist(df=df1)
    print('nans')
    print(nans_lst[1:])
    print('zeros')
    print(zeros_lst[1:])
    print('#####################')
    print()

# Downsample after applying anti-aliasing filter

In [ ]:
factor = int(original_sample_freq / new_sample_freq)
print(factor)

for x in df_name_lst:

    df1 = locals()[x]

    # get df with time values
    df_x = df1.iloc[::factor, :].time
    df_x.reset_index(drop=True, inplace=True)
    df_x = pd.DataFrame(df_x)

    for y in chan_to_keep[1:]:    
        arr = Preprocessing.downsample_with_anti_aliasing(arr=df1[y], factor=factor, ftype='iir')
        df_x[y] = arr.tolist()

    exec(x + " = df_x")
    print(df_x)

## Determine percentage missing values

In [ ]:
for x in df_name_lst:
    print(x)
    df1 = locals()[x]
    nans_lst, zeros_lst = Preprocessing.percent_nans_zeros_tolist(df=df1)
    print('nans')
    print(nans_lst[1:])
    print('zeros')
    print(zeros_lst[1:])
    print('#####################')
    print()

# Remove dropped packets (errors in the hardware recording)

In [ ]:
if use_dropped_packet_removal:
    
    df_packet_drop = pd.DataFrame()
    
    removed_pct_lst = []
    cond_lst = []
    chan_lst = []

    for x in df_name_lst:
 
        df_x = locals()[x]
        
        for y in chan_to_keep[1:]:

            df_x[y], removed_pct = Preprocessing.remove_dropped_packets_zerofill(arr=df_x[y], thres=100000)
            removed_pct_lst.append(removed_pct)
            cond_lst.append(x)
            chan_lst.append(y)
            
        exec(x + ' = df_x')
 
    df_packet_drop['cond'] = cond_lst
    df_packet_drop['chan'] = chan_lst
    df_packet_drop['removed_pct'] = removed_pct_lst
    
    print(df_packet_drop)

## Determine percentage missing values

In [ ]:
for x in df_name_lst:
    print(x)
    df1 = locals()[x]
    nans_lst, zeros_lst = Preprocessing.percent_nans_zeros_tolist(df=df1)
    print('nans')
    print(nans_lst[1:])
    print('zeros')
    print(zeros_lst[1:])
    print('#####################')
    print()

# Plot raw signals

In [ ]:
for x in chan_to_keep[1:]:
    plt.figure(figsize=(15,5))
    plt.title(x)
    plt.plot(df_base['time'], df_base[x], label='base')
    plt.plot(df_feed['time'], df_feed[x], label='feed')
    plt.legend(loc='upper right')

# Apply filters

## If band pass

In [ ]:
if filter_type == 'bandpass':
    
    for x in df_name_lst:
        df_x = locals()[x]
        for y in chan_to_keep[1:]:
            df_x[y] = Preprocessing.bandpass_butter_filter(arr=df_x[y], order=2, sample_freq=new_sample_freq, 
                                                           lowcut=lo_cutoff, highcut=hi_cutoff)
        exec(x + ' = df_x')

    for x in chan_to_keep[1:]:
        plt.figure(figsize=(15,5))
        plt.title(x)
        plt.plot(df_base['time'], df_base[x], label='base')
        plt.plot(df_feed['time'], df_feed[x], label='feed')
        plt.legend(loc='upper right')

## If high pass

In [ ]:
if (filter_type == 'butter_high_low') or (filter_type == 'butter_high_savgol_low') or (filter_type == 'butter_high_only'):
    
    for x in df_name_lst:
        df_x = locals()[x]
        for y in chan_to_keep[1:]:
            df_x[y] = Preprocessing.highpass_butter_filter(arr=df_x[y], order=2, sample_freq=new_sample_freq, cutoff_freq=lo_cutoff)
        exec(x + ' = df_x')

    for x in chan_to_keep[1:]:
        plt.figure(figsize=(15,5))
        plt.title(x)
        plt.plot(df_base['time'], df_base[x], label='base')
        plt.plot(df_feed['time'], df_feed[x], label='feed')
        plt.legend(loc='upper right')

## If low pass

In [ ]:
if filter_type == 'butter_high_low':
        
    for x in df_name_lst:
        df_x = locals()[x]
        for y in chan_to_keep[1:]:
            df_x[y] = Preprocessing.lowpass_butter_filter(arr=df_x[y], order=2, sample_freq=new_sample_freq, 
                                                           cutoff_freq=hi_cutoff)
        exec(x + ' = df_x')
        
    for x in chan_to_keep[1:]:
        plt.figure(figsize=(15,5))
        plt.title(x)
        plt.plot(df_base['time'], df_base[x], label='base')
        plt.plot(df_feed['time'], df_feed[x], label='feed')
        plt.legend(loc='upper right')

if filter_type == 'butter_high_savgol_low':
            
    for x in df_name_lst:
        df_x = locals()[x]
        for y in chan_to_keep[1:]:
            arr = Preprocessing.savitzky_golay_filter(arr=df_x[y].values, win_len=sg_len, order=sg_order)
            df_x[y] = pd.Series(arr)
        exec(x + ' = df_x')

    for x in col_names[1:]:
        plt.figure(figsize=(15,5))
        plt.title(x)
        plt.plot(df_base['time'], df_base[x], label='base')
        plt.plot(df_feed['time'], df_feed[x], label='feed')
        plt.legend(loc='upper right')

## Determine percentage missing values

In [ ]:
for x in df_name_lst:
    print(x)
    df1 = locals()[x]
    nans_lst, zeros_lst = Preprocessing.percent_nans_zeros_tolist(df=df1)
    print('nans')
    print(nans_lst[1:])
    print('zeros')
    print(zeros_lst[1:])
    print('#####################')
    print()

# Artifact removal > 2 mV

In [ ]:
if use_artifact_removal:
    
    df_artifact_remove = pd.DataFrame()
    
    removed_pct_lst = []
    cond_lst = []
    chan_lst = []

    for x in df_name_lst:
 
        df_x = locals()[x]
        
        for y in chan_to_keep[1:]:
            
            # apply filter
            df_x[y], removed_pct = Preprocessing.artifacts_removal(arr=df_x[y], thres=2000, 
                                                                   pts_pre=pre_post_pts, 
                                                                   pts_post=pre_post_pts, 
                                                                   interpolate=True,
                                                                   fill_frontend=fill_frontend)
            removed_pct_lst.append(removed_pct)
            cond_lst.append(x)
            chan_lst.append(y)
        print(df_x)   
        exec(x + ' = df_x')
 
    df_artifact_remove['cond'] = cond_lst
    df_artifact_remove['chan'] = chan_lst
    df_artifact_remove['removed_pct'] = removed_pct_lst
    
    print(df_artifact_remove)

    for x in chan_to_keep[1:]:
        plt.figure(figsize=(15,5))
        plt.plot(df_base['time'], df_base[x], label='base')
        plt.plot(df_feed['time'], df_feed[x], label='feed')
        plt.legend(loc='upper right')

## Determine percentage missing values

In [ ]:
for x in df_name_lst:
    print(x)
    df1 = locals()[x]
    nans_lst, zeros_lst = Preprocessing.percent_nans_zeros_tolist(df=df1)
    print('nans')
    print(nans_lst[1:])
    print('zeros')
    print(zeros_lst[1:])
    print('#####################')
    print()

# Plots: power spectral density (from 3 to 15 cpm)

In [ ]:
df_summary = pd.DataFrame()

for x in df_name_lst:
    
    print(x)

    df_x = locals()[x]
    
    for y in chan_to_keep[1:]:
        df_final = Plotting.psd(df_x[y], fs=new_sample_freq, xlim=[0,0.25], units='V', title=y, 
                                tot_pwr_lst=[0.05, 0.25], brad_pwr_lst=[0.05, 0.127], norm_pwr_lst=[0.127, 0.19], tach_pwr_lst=[0.19, 0.25])
        df_final['cond'] = x
        df_final['chan'] = y
        df_summary = pd.concat([df_summary, df_final])
    exec(x + '_summary = df_summary')

# change the order of columns and drop a column
df_summary = df_summary[['cond', 'chan', 'tot_power', 'brad_power', 'norm_power', 'tach_power', 'brad_domf', 'norm_domf', 'tach_domf']]
    
print(df_summary)

print('#########################################################')
print()

# Plot spectrograms

In [ ]:
for x in df_name_lst:
    
    print(x)

    df_x = locals()[x]
    
    for y in chan_to_keep[1:]:
        Plotting.spect(arr=df_x[y], sample_freq=new_sample_freq, y_max_freq=1.5*hi_cutoff, plot_title=y)
        
    print('#########################################################')
    print()

# make summary table

In [ ]:
# use deep = true to make sure that original will not be changed when modifying copy
df_copy = df_summary.copy(deep=True)

# add columns
df_copy['packet_remove_pct'] = df_packet_drop['removed_pct'].values
df_copy['art_remove_pct'] = df_artifact_remove['removed_pct'].values
df_copy['animal'] = pd.Series(animal, index=df_copy.index)
df_copy['new_sample_freq'] = pd.Series(new_sample_freq, index=df_copy.index)
df_copy['filter_type'] = pd.Series(filter_type, index=df_copy.index)
df_copy['lo_cutoff'] = pd.Series(lo_cutoff, index=df_copy.index)
df_copy['hi_cutoff'] = pd.Series(hi_cutoff, index=df_copy.index)
df_copy['remove_drop_packs'] = pd.Series(use_dropped_packet_removal, index=df_copy.index)
df_copy['remove_artifact'] = pd.Series(use_artifact_removal, index=df_copy.index)

# reorder columns
df_copy = df_copy[['animal', 'cond', 'chan', 'new_sample_freq', 'filter_type', 'lo_cutoff', 'hi_cutoff', 
                   'remove_drop_packs', 'remove_artifact', 'tot_power', 'brad_power', 
                   'norm_power', 'tach_power', 'brad_domf', 'norm_domf', 'tach_domf', 'packet_remove_pct', 'art_remove_pct']]

# replace characters
df_copy['cond'] = df_copy.cond.str.replace('df_', '')

print(df_copy)

# Write to file
results_file = 'summary.csv'

# make a results file
if not os.path.exists(results_file):
    os.mknod(results_file)
    df_copy.to_csv(results_file)  
elif os.path.exists(results_file):
    df_copy.to_csv(results_file, mode='a', header=False)

# make filtered files

In [ ]:
if not os.path.exists('final_filt_data'):
    os.mkdir('final_filt_data')
    
for x in df_name_lst:

    df_x = locals()[x]    
    
    # make csv
    df_x.to_csv('final_filt_data/' + animal + '_' + x + '.csv', index=False)
    
    # make pickle
    f = open('final_filt_data/' + animal + '_' + x + '.pckl', 'wb')
    pickle.dump(df_x, f)
    f.close()